In [11]:
# ==============================================================================
# 1. SETUP - INSTALL AND IMPORT LIBS
# ==============================================================================
!pip install langgraph langchain_core==0.1.52 --quiet

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import TypedDict, List

# ==============================================================================
# 2. DEFINE THE STATE AND WORKFLOW LOGIC
# ==============================================================================
class TimeTravelState(TypedDict):
    value: int
    history: List[str]

def step_a(state: TimeTravelState) -> dict:
    print("⚡ Executing Step A: value + 1")
    new_value = state["value"] + 1
    return {"value": new_value, "history": state["history"] + [f"A (value={new_value})"]}

def step_b(state: TimeTravelState) -> dict:
    print("⚡ Executing Step B: value * 2")
    new_value = state["value"] * 2
    return {"value": new_value, "history": state["history"] + [f"B (value={new_value})"]}

def step_c(state: TimeTravelState) -> dict:
    print("⚡ Executing Step C: value - 5")
    new_value = state["value"] - 5
    return {"value": new_value, "history": state["history"] + [f"C (value={new_value})"]}

# ==============================================================================
# 3. BUILD AND COMPILE THE WORKFLOW
# ==============================================================================
def create_time_travel_workflow():
    workflow = StateGraph(TimeTravelState)
    workflow.add_node("a", step_a)
    workflow.add_node("b", step_b)
    workflow.add_node("c", step_c)

    workflow.add_edge(START, "a")
    workflow.add_edge("a", "b")
    workflow.add_edge("b", "c")
    workflow.add_edge("c", END)

    # ✅ return both compiled app and memory (checkpointer)
    memory = SqliteSaver.from_conn_string(":memory:")
    return workflow.compile(checkpointer=memory), memory

# ==============================================================================
# 4. RUN THE WORKFLOW AND DEMONSTRATE TIME TRAVEL
# ==============================================================================
if __name__ == "__main__":
    app, memory = create_time_travel_workflow()

    # ✅ Must wrap thread_id inside "configurable"
    thread_config = {"configurable": {"thread_id": "tt-789"}}
    initial_state = {"value": 10, "history": []}

    print("=== Part 1: Running the full workflow to generate history ===")
    final_state = app.invoke(initial_state, config=thread_config)
    print(f"\n✅ Final State: {final_state}\n")

    print("=== Part 2: Time Traveling through the saved checkpoints ===")
    checkpoints = list(memory.list(config=thread_config))  # ✅ use memory, not app

    if not checkpoints:
        print("⚠️ No checkpoints found.")
    else:
        for i, checkpoint in enumerate(checkpoints):
            # The timestamp is available directly in the checkpoint object
            config_at_checkpoint = {
                "configurable": {
                    "thread_id": "tt-789",
                    "checkpoint_ts": checkpoint[1] # Use checkpoint[1] for timestamp
                }
            }
            state_at_checkpoint = memory.get(config=config_at_checkpoint)
            # Access the actual state values within 'channel_values'
            values = state_at_checkpoint['channel_values'] if state_at_checkpoint else {}

            print(f"\n⏳ Checkpoint {i+1} (ts={checkpoint[1]}):") # Use checkpoint[1] for timestamp
            print(f"  Value: {values.get('value')}")
            print(f"  History: {values.get('history')}")

=== Part 1: Running the full workflow to generate history ===
⚡ Executing Step A: value + 1
⚡ Executing Step B: value * 2
⚡ Executing Step C: value - 5

✅ Final State: {'value': 17, 'history': ['A (value=11)', 'B (value=22)', 'C (value=17)']}

=== Part 2: Time Traveling through the saved checkpoints ===

⏳ Checkpoint 1 (ts={'channel_values': {'c': 'c', 'history': ['A (value=11)', 'B (value=22)', 'C (value=17)'], 'value': 17}, 'channel_versions': {'__start__': 1, 'a': 3, 'b': 4, 'c': 5, 'history': 5, 'start:a': 2, 'value': 5}, 'id': '1f09f813-01af-648b-8003-44ed74ed4e76', 'ts': '2025-10-02T11:16:05.515562+00:00', 'v': 1, 'versions_seen': {'__start__': {'__start__': 1}, 'a': {'start:a': 2}, 'b': {'a': 3}, 'c': {'b': 4}}}):
  Value: 17
  History: ['A (value=11)', 'B (value=22)', 'C (value=17)']

⏳ Checkpoint 2 (ts={'channel_values': {'b': 'b', 'history': ['A (value=11)', 'B (value=22)'], 'value': 22}, 'channel_versions': {'__start__': 1, 'a': 3, 'b': 4, 'history': 4, 'start:a': 2, 'value'